In [1]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [2]:
from data_sources.clob import CLOBDataSource

# Get trading rules and candles
clob = CLOBDataSource()

In [3]:
# Constants
CONNECTOR_NAME = "binance"
INTERVAL = "15m"
DAYS = 20


# Features configuration
VOLATILITY_WINDOW = 100
TREND_SHORT_WINDOW = 20
TREND_LONG_WINDOW = 100
VOLUME_SHORT_WINDOW = 20
VOLUME_LONG_WINDOW = 100

# Download data
- Get trading rules
- Get candles for the last x days

In [4]:
import asyncio

trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_pairs = trading_rules.filter_by_quote_asset("FDUSD")\
    .filter_by_min_notional_size(Decimal("5"))\
    .get_all_trading_pairs()

2024-08-14 20:48:44,016 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x163c506d0>
2024-08-14 20:48:44,017 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x119ef15a0>, 65638.559950083)]']
connector: <aiohttp.connector.TCPConnector object at 0x163c50700>


In [5]:
tasks = [clob.get_candles_last_days(
    connector_name=CONNECTOR_NAME,
    trading_pair=trading_pair,
    interval=INTERVAL,
    days=DAYS,
) for trading_pair in trading_pairs]

candles = await asyncio.gather(*tasks)
candles = {trading_pair: candle for trading_pair, candle in zip(trading_pairs, candles)}

2024-08-14 20:48:46,596 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x165395270>
2024-08-14 20:48:46,597 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x164f1f0a0>, 65641.132635416)]']
connector: <aiohttp.connector.TCPConnector object at 0x1653952a0>
2024-08-14 20:48:47,373 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x16536ddb0>
2024-08-14 20:48:47,374 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x164fa56c0>, 65641.910506958)]']
connector: <aiohttp.connector.TCPConnector object at 0x16536dde0>
2024-08-14 20:48:47,380 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x163d3efb0>
2024-08-14 20:48:47,381 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x164

In [6]:
from features.candles.trend import TrendConfig
from features.candles.volatility import VolatilityConfig
from features.candles.volume import VolumeConfig
from research_notebooks.dneitor.utils import generate_report

report = generate_report(
    candles=candles,
    volatility_config=VolatilityConfig(window=VOLATILITY_WINDOW),
    trend_config=TrendConfig(short_window=TREND_SHORT_WINDOW, long_window=TREND_LONG_WINDOW),
    volume_config=VolumeConfig(short_window=VOLUME_SHORT_WINDOW, long_window=VOLUME_LONG_WINDOW))
report

,trading_pair,mean_volatility,mean_natr,mean_bb_width,latest_trend,average_volume_per_hour,normalized_score
0,ARKM-FDUSD,0.00713384,0.00720998,10.19875338,-0.86624251,5409.21190703,0.00008372
1,CYBER-FDUSD,0.00630152,0.00508397,9.38722006,-0.84539638,1250.63672467,0.0000178
2,FLOKI-FDUSD,0.00690575,0.00904054,9.51981718,-0.76565808,21921.07878097,0.00031699
3,WIF-FDUSD,0.00863572,0.0117553,13.12701323,0.27402366,95176.2179191,0.00189738
4,SUPER-FDUSD,0.00695593,0.00519959,11.30814214,-0.45077336,791.82914268,0.00001357
...,...,...,...,...,...,...,...
89,AR-FDUSD,0.00735165,0.00568418,10.75320414,-0.66627834,4961.62955373,0.00008091
90,PIXEL-FDUSD,0.00690456,0.00585629,10.81238924,0.60210014,3309.8068652,0.00005427
91,ACE-FDUSD,0.00951952,0.00930991,13.9075923,-0.36190228,4984.03526816,0.00010517
92,ENA-FDUSD,0.00699283,0.00820541,11.22890391,-0.11450747,41749.21944815,0.00071148


In [7]:
from research_notebooks.dneitor.utils import filter_top_markets

TOP_X_MARKETS = 10  # Number of top markets to select
MIN_VOLUME_USD = 2000  # Minimum volume in USD
MIN_NATR = 0.008  # Minimum ATR
TREND_THRESHOLD = -0.5  # Trend threshold

top_markets = filter_top_markets(report_df=report, top_x=TOP_X_MARKETS, min_volume_usd=MIN_VOLUME_USD, min_natr=MIN_NATR, trend_threshold=TREND_THRESHOLD)
top_markets

,trading_pair,mean_volatility,mean_natr,mean_bb_width,latest_trend,average_volume_per_hour,normalized_score
49,SOL-FDUSD,0.00595561,0.00893356,8.68034853,0.00970546,17917442.64608948,0.2363222
83,PEPE-FDUSD,0.00730638,0.01072627,10.64426857,0.05615689,211227.42629836,0.00341602
3,WIF-FDUSD,0.00863572,0.0117553,13.12701323,0.27402366,95176.2179191,0.00189738
53,NOT-FDUSD,0.00715373,0.0098299,10.75858791,-0.40061164,95522.78778295,0.00156082
63,BOME-FDUSD,0.00792702,0.0109994,12.10217303,0.66184623,43785.54740658,0.00080478
65,NEAR-FDUSD,0.00632563,0.00841817,9.31496342,-0.02702003,51660.14320063,0.00073082
92,ENA-FDUSD,0.00699283,0.00820541,11.22890391,-0.11450747,41749.21944815,0.00071148
29,1000SATS-FDUSD,0.00913956,0.01248341,13.80358331,0.00696545,28984.52918517,0.00060762
40,BONK-FDUSD,0.0072864,0.00941468,9.97378365,-0.01775732,36602.37855511,0.00055451
66,BANANA-FDUSD,0.00932825,0.01116695,12.65190877,-0.10955719,26123.81897332,0.00050192


In [8]:
from research_notebooks.dneitor.utils import generate_config, dump_dict_to_yaml


TOTAL_AMOUNT = 3000  # General total amount for all markets
MIN_AMOUNT_PER_MARKET = 200  # Minimum amount per market
ACTIVATION_BOUNDS = 0.002  # Input activation bounds
MAX_OPEN_ORDERS = 1  # Input max open orders for each market

TAKE_PROFIT_MULTIPLIER = 0.3  # Multiplier for take profit based on NATR
AMOUNTS_QUOTE_PCT = [0.1, 0.1, 0.1, 0.1, 0.2, 0.2]  # Weights for each cluster

strategy_config = generate_config(
    id="dneitor-binance_0.4",
    connector_name=CONNECTOR_NAME,
    candles=candles,
    top_markets=top_markets,
    total_amount=TOTAL_AMOUNT,
    amounts_quote_pct=AMOUNTS_QUOTE_PCT,
    activation_bounds=ACTIVATION_BOUNDS,
    take_profit_multiplier=TAKE_PROFIT_MULTIPLIER,
    max_open_orders=MAX_OPEN_ORDERS,
    min_amount_per_market=MIN_AMOUNT_PER_MARKET,
)
dump_dict_to_yaml("configs/", strategy_config)